In [11]:
import h5py
from Bio import SeqIO, AlignIO
import io
import numpy as np

pfam_path = '/home/cactuskid13/struct_data/pfam/Pfam-A.full'
import gzip

def yield_alns(pfam_path , verbose = False):    
    skip = False
    with open(pfam_path, 'rb') as f:
        aln= ''
        acc = None
        for i,l in enumerate( f ):
            if verbose == True:
                if i < 1000:
                    print(l)
            aln+=l.decode()
            if acc is None and 'AC' in l.decode():
                acc = l.decode().split()[2]     #temporary solution: skipping the file causing a problem (index 4226)
                if acc == 'PF00823':
                    skip = True
            if l == b'//\n':
                if skip:
                    skip = False
                else:
                    yield acc, AlignIO.read(io.StringIO(aln), "stockholm")
                aln = ''
                acc = None


In [15]:
with h5py.File('Pfam-A.full' +'.h5', 'w') as hf:
    i = 0
    for pfam_id, msa in yield_alns(pfam_path):
        i += 1
        #print(i)
        align_array = np.array([ list(rec.upper()) for rec in msa], np.character)
        hf.create_dataset(pfam_id,  data=align_array)
    

/home/osboxes/miniconda3/envs/structML/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Converting `np.character` to a dtype is deprecated. The current result is `np.dtype(np.str_)` which is not strictly correct. Note that `np.character` is generally deprecated and 'S1' should be used.
  
